In [1]:
%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import get_state as getstate
import utility as util
import plotfig as fig
import ground_state as gs
import lanczos
import time
start_time = time.time()
M_PI = math.pi
                  
#####################################
def compute_Aw_main(A,ep,tpd,tpp,pds,pdp,pps,ppp,Upp,\
                    d_double,p_double,U, S_val, Sz_val, AorB_sym): 
    if Norb==7:
        fname = 'ep'+str(ep)+'_tpd'+str(tpd)+'_tpp'+str(tpp) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_tpp'+str(tpp)+'_Mc'+str(Mc)+'_eta'+str(eta)
    elif Norb==9 or Norb==10 or Norb==11:
        fname = 'ep'+str(ep)+'_pdp'+str(pdp)+'_pps'+str(pps)+'_ppp'+str(ppp) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta)
        flowpeak = 'Norb'+str(Norb)+'_pps'+str(pps)+'_ppp'+str(ppp)+'_Mc'+str(Mc)+'_eta'+str(eta)
                
    w_vals = np.arange(pam.wmin, pam.wmax, pam.eta)
    Aw = np.zeros(len(w_vals))
    Aw_dd_total = np.zeros(len(w_vals))
    Aw_d8_total = np.zeros(len(w_vals))

    # set up H0
    if Norb==7:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,tpd,tpp,0,0,0,0)
    elif Norb==9 or Norb==11:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,0,0,pds,pdp,pps,ppp)
            
    T_pd   = ham.create_tpd_nn_matrix(VS,tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac)
    T_pp   = 0
#     T_pp   = ham.create_tpp_nn_matrix(VS,tpp_nn_hop_fac)  
    Esite  = ham.create_edep_diag_matrix(VS,A,ep)
    
    H0 = T_pd + T_pp + Esite  
    
    '''
    Below probably not necessary to do the rotation by multiplying U and U_d
    the basis_change.py is only for label the state as singlet or triplet
    and assign the interaction matrix
    '''
    if pam.if_H0_rotate_byU==1:
        H0_new = U_d.dot(H0.dot(U))
    
    clf()

    if Norb==7 or Norb==9 or Norb==10 or Norb==11:     
        Hint = ham.create_interaction_matrix_ALL_syms(VS,d_double,p_double,S_val, Sz_val,AorB_sym,A,Upp)
        if pam.if_H0_rotate_byU==1:
            H = H0_new + Hint 
        else:
            H = H0 + Hint 
        H.tocsr()

        ####################################################################################
        # compute GS only for turning on full interactions
        if pam.if_get_ground_state==1:
            vals, vecs, wgt_d8, wgt_d9L, wgt_d10L2 = gs.get_ground_state(H, VS, S_val, Sz_val)
            if Norb==8:
                util.write_GS('Egs_'+flowpeak+'.txt',A,ep,tpd,vals[0])
                util.write_GS_components('GS_weights_'+flowpeak+'.txt',A,ep,tpd,wgt_d8, wgt_d9L, wgt_d10L2)
            elif Norb==10 or Norb==11 or Norb==12:
                util.write_GS2('Egs_'+flowpeak+'.txt',A,ep,pds,pdp,vals[0])
                util.write_GS_components2('GS_weights_'+flowpeak+'.txt',A,ep,pds,pdp,wgt_d8, wgt_d9L, wgt_d10L2)
            
        #########################################################################
        '''
        Compute A(w) for various states
        '''
#         # compute d8
#         fig.compute_Aw_d8_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, A, w_vals, "Aw_d8_sym_", fname)
        
#         # compute d9L
#         b1L_state_indices, a1L_state_indices, b1L_state_labels, a1L_state_labels \
#                 = getstate.get_d9L_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, b1L_state_indices, b1L_state_labels, "Aw_b1L_", fname)
#         fig.compute_Aw1(H, VS, w_vals, a1L_state_indices, a1L_state_labels, "Aw_a1L_", fname)
        
#         # compute d10L2
#         d10L2_state_indices, d10L2_state_labels = getstate.get_d10L2_state_indices(VS, S_val, Sz_val)
#         fig.compute_Aw1(H, VS, w_vals, d10L2_state_indices, d10L2_state_labels, "Aw_d10L2_", fname)
        
##########################################################################
if __name__ == '__main__': 
    Mc  = pam.Mc
    print ('Mc=',Mc)

    Norb = pam.Norb
    eta  = pam.eta
    ed   = pam.ed

    As = pam.As
    B  = pam.B
    C  = pam.C
    
    # set up VS
    VS = vs.VariationalSpace(Mc)
    basis.count_VS(VS)
    
    d_double, p_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    if pam.basis_change_type=='all_states':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix(VS,d_double)
        if pam.if_print_VS_after_basis_change==1:
            basis.print_VS_after_basis_change(VS,S_val,Sz_val)
    elif pam.basis_change_type=='d_double':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix_d_double(VS,d_double)

    U_d = (U.conjugate()).transpose()
    # check if U if unitary
    #checkU_unitary(U,U_d)
    
    if Norb==7:
        for tpd in pam.tpds:
            for ep in pam.eps:               
                for A in pam.As:
                    util.get_atomic_d8_energy(A,B,C)
                    for tpp in pam.tpps:
                        for Upp in pam.Upps:
                            print ('===================================================')
                            print ('A=',A, 'ep=', ep, ' tpd=',tpd,' tpp=',tpp,' Upp=',Upp )
                            compute_Aw_main(A,ep,tpd,tpp,0,0,0,0,Upp,\
                                            d_double,p_double,U, S_val, Sz_val, AorB_sym)
    elif Norb==9 or Norb==10 or Norb==11:
        pps = pam.pps
        ppp = pam.ppp
        for ii in range(0,len(pam.pdps)):
            pds = pam.pdss[ii]
            pdp = pam.pdps[ii]
            for ep in pam.eps:
                for A in pam.As:
                    util.get_atomic_d8_energy(A,B,C)
                    for Upp in pam.Upps:
                        print ('===================================================')
                        print ('A=',A, 'ep=',ep,' pds=',pds,' pdp=',pdp,' pps=',pps,' ppp=',ppp,' Upp=',Upp)
                        compute_Aw_main(A,ep,0,0,pds,pdp,pps,ppp,Upp,\
                                        d_double,p_double,U, S_val, Sz_val, AorB_sym)      
                        
    print("--- %s seconds ---" % (time.time() - start_time))

Populating the interactive namespace from numpy and matplotlib
Ni_orbs =  ['d3z2r2', 'dx2y2', 'dxy', 'dxz', 'dyz']
O1_orbs =  ['px']
O2_orbs =  ['py']
compute A(w) for symmetries =  ['1A1', '1B1', '3B1', '1A2', '3A2', '1E', '3E']
Mc= 8
VS.dim =  4761
No. of states with count_upup, count_updn, count_dnup, count_dndn: 0 2425 2336 0
len(d_list) 25
len(p_list) 64
E_1S =  0.0
E_1G =  0.0
E_1D =  0.0
E_3P =  0.0
E_3F =  0.0
A= 0.0 ep= 0.0  tpd= 1.5  tpp= 0.55  Upp= 0
start create_tpd_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-6.         -4.73205081 -4.73205081 -3.46410162 -3.         -3.
 -3.         -3.         -3.         -3.        ]
eigenvalue =  -6.000000000000021
Compute the weights in GS (lowest Aw peak)
no e-h state  px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.015625000000000007
no e-h state  px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.015625000000000014
no e-h state  px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight = 

E_1S =  0.0
E_1G =  0.0
E_1D =  0.0
E_3P =  0.0
E_3F =  0.0
A= 0.0 ep= 4.0  tpd= 1.5  tpp= 0.55  Upp= 0
start create_tpd_nn_matrix
start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-3.21110255 -2.25130259 -2.25130259 -1.60555128 -1.60555128 -1.60555128
 -1.60555128 -1.60555128 -1.60555128 -1.29150262]
eigenvalue =  -3.211102550927981
Compute the weights in GS (lowest Aw peak)
no e-h state  px -1 0 0 px -1 0 0 S= 0 Sz= 0 , weight =  0.003098311175653888
no e-h state  px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.003098311175653891
no e-h state  px -1 0 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.003098311175653905
no e-h state  py 0 -1 0 py 0 -1 0 S= 0 Sz= 0 , weight =  0.0030983111756539075
no e-h state  dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.04326923076923066
no e-h state  dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.04326923076923076
no e-h state  dx2y2 0 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.043269230769230844
no e-h state  d

start create_edep_diag_matrix
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[-1.66190379 -1.12245452 -1.12245452 -0.83095189 -0.83095189 -0.83095189
 -0.83095189 -0.83095189 -0.83095189 -0.58300524]
eigenvalue =  -1.6619037896906035
Compute the weights in GS (lowest Aw peak)
no e-h state  dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.016544117647058855
no e-h state  dx2y2 -1 0 0 px 0 0 0 S= 0 Sz= 0 , weight =  0.016544117647058827
no e-h state  dx2y2 0 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.016544117647058855
no e-h state  dx2y2 0 -1 0 py 0 0 0 S= 0 Sz= 0 , weight =  0.016544117647058834
no e-h state  dx2y2 0 0 0 dx2y2 0 0 0 S= 0 Sz= 0 , weight =  0.8625699922680371
no e-h state  dx2y2 0 0 0 py 0 1 0 S= 0 Sz= 0 , weight =  0.016544117647058834
no e-h state  dx2y2 0 0 0 py 0 1 0 S= 0 Sz= 0 , weight =  0.016544117647058834
no e-h state  dx2y2 0 0 0 px 1 0 0 S= 0 Sz= 0 , weight =  0.016544117647058827
no e-h state  dx2y2 0 0 0 px 1 0 0 S= 0 Sz= 0 , weight =  0

<Figure size 432x288 with 0 Axes>